In [ ]:
# import library
library(dplyr)

In [ ]:
# read datasets
matches = read.csv("matches.csv")
deli = read.csv("deliveries.csv")

# Question 1:

In [ ]:
# teams playing today's match
teams = c("Rajasthan Royals", "Kolkata Knight Riders")

In [ ]:
# all matches between KKR and RR
kkr_rr = subset(matches, (team1 %in% teams & team2 %in% teams))

head(kkr_rr)

In [ ]:
kkr_rr$winner

In [ ]:
# victories of the teams against each other
count(kkr_rr, winner)

In [ ]:
# matches of RR
rr = subset(matches, (team1 == 'Rajasthan Royals'| team2 == 'Rajasthan Royals'))

In [ ]:
# total no. of matches played by RR
nrow(rr)

In [ ]:
# winners in those matches
count(rr, winner)

In [ ]:
# win ratio of RR
75/147

In [ ]:
# matches of KKR
kkr = subset(matches, (team1 == 'Kolkata Knight Riders'| team2 == 'Kolkata Knight Riders'))

In [ ]:
# total no. of matches played by KKR
nrow(kkr)

In [ ]:
# winners in those matches
count(kkr, winner)

In [ ]:
# win ratio of KKR
92/178

# Question 2:

In [ ]:
# total wickets taken in each IPL match
total_wickets = subset(deli, player_dismissed != '') %>% group_by(match_id) %>% summarise(wickets = n())

head(total_wickets)

In [ ]:
# histogram
hist(total_wickets$wickets, freq=TRUE, breaks=c(0, 5, 10, 15, max(total_wickets$wickets)))

In [ ]:
# all deliveries in KKR vs RR matches
kkr_rr_deli = subset(deli, (bowling_team %in% teams & batting_team %in% teams))

head(kkr_rr_deli)

In [ ]:
# total wickets taken in KKR vs RR matches
wickets_kkr_rr = subset(kkr_rr_deli, player_dismissed != '') %>% group_by(match_id) %>% summarise(wickets = n())

# histogram
hist(wickets_kkr_rr$wickets, freq=TRUE, breaks=c(0, 5, 10, 15, max(wickets_kkr_rr$wickets)))

In [ ]:
# all deliveries faced by KKR
kkr_bat = subset(deli, batting_team == 'Kolkata Knight Riders')

# total wickets lost by KKR in each match
wickets_kkr = subset(kkr_bat, player_dismissed != '') %>% group_by(match_id) %>% summarise(wickets = n())

#statistical measures
summary(wickets_kkr$wickets)

In [ ]:
# all deliveries faced by RR
rr_bat = subset(deli, batting_team == 'Rajasthan Royals')

# total wickets lost by RR in each match
wickets_rr = subset(rr_bat, player_dismissed != '') %>% group_by(match_id) %>% summarise(wickets = n())

#statistical measures
summary(wickets_rr$wickets)

# Question 3:

In [ ]:
# batsmen playing today's match
batsmen = c('RV Uthappa', 'BA Stokes', 'SV Samson', 'SPD Smith', 'JC Buttler', 'R Parag',
           'S Gill', 'N Rana', 'RA Tripathi', 'KD Karthik', 'EJG Morgan', 'R Singh', 'SP Narine')

In [ ]:
# function to get the runs ratio for the batsman
get_runs_ratio = function(deli_data, b_name){
    
    data = distinct(subset(deli_data, with(deli_data, grepl(b_name, batsman))))
    runs_notb = subset(data, batsman_runs %in% c(1, 2, 3)) %>%  group_by(match_id) %>% summarise(runs_nb=sum(batsman_runs))

    runs_b = subset(data, batsman_runs %in% c(4, 6)) %>%  group_by(match_id) %>% summarise(runs_b=sum(batsman_runs))

    total_runs = data %>%  group_by(match_id) %>% summarise(total_runs=sum(batsman_runs))

    
    balls = data %>%  group_by(match_id) %>% summarise(balls=n())

    df1 = merge(runs_notb, runs_b, by='match_id', all=TRUE)
    df2 = merge(total_runs, balls, by='match_id', all=TRUE)
    df = merge(df1, df2, all=TRUE)
    df[is.na(df)] = 0
    df = subset(df, balls>10)
    df$ratio = (df$runs_b - df$runs_nb)/df$total_runs
    return (summary(df$ratio))
}

In [ ]:
for (b_name in batsmen){
    print(b_name)
    print(get_runs_ratio(deli, b_name))
}

In [ ]:
for (b_name in batsmen){
    print(b_name)
    print(get_runs_ratio(subset(deli, match_id >11000), b_name))
}

# Question 4:

In [ ]:
archer = subset(deli, with(deli, grepl('J Archer', bowler)))

In [ ]:
archer = distinct(archer)

In [ ]:
runs = archer %>%  group_by(match_id) %>% summarise(runs=sum(wide_runs, noball_runs, batsman_runs))

In [ ]:
overs = archer %>%  group_by(match_id) %>% summarise(overs=length(unique(over)))

In [ ]:
df = merge(runs, overs)

df$economy = df$runs/df$overs

In [ ]:
hist(df$economy, freq=TRUE, breaks=c(0, 7.5, max(df$economy)))

In [ ]:
boundaries = subset(archer, batsman_runs %in% c(4, 6)) %>%  group_by(match_id) %>% summarise(boundaries=n())

hist(boundaries$boundaries, freq=TRUE, breaks=c(0, 4, max(boundaries$boundaries)))

In [ ]:
kkr_match_ids = unique(subset(archer, batting_team=='Kolkata Knight Riders')$match_id)

In [ ]:
subset(df, match_id %in% kkr_match_ids)

In [ ]:
subset(boundaries, match_id %in% kkr_match_ids)

# Question 5:

In [ ]:
bowlers = c('V Chakravarthy', 'L Ferguson', 'PJ Cummins', 'P Krishna', 'S Mavi', 'SP Narine',
           'J Archer', 'VR Aaron', 'S Gopal', 'R Tewatia', 'AS Rajpoot')

In [ ]:
get_economy = function(deli_data, b_name){
    data = distinct(subset(deli_data, with(deli_data, grepl(b_name, bowler))))
    runs = data %>%  group_by(match_id) %>% summarise(runs=sum(wide_runs, noball_runs, batsman_runs))
    overs = data %>%  group_by(match_id) %>% summarise(overs=length(unique(over)))
    df = merge(runs, overs)
    df = subset(df, overs>1)
    df$economy = df$runs/df$overs
    return (summary(df$economy))
}

In [ ]:
for (b_name in bowlers){
    print(b_name)
    print(get_economy(deli, b_name))
}

In [ ]:
for (b_name in bowlers){
    print(b_name)
    print(get_economy(subset(deli, match_id>11000), b_name))
}